In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/DS/Time_series_project_3

/content/drive/MyDrive/DS/Time_series_project_3


In [ ]:
! pip install holidays
! pip install tsfresh
! pip install fcbf
! pip install skfeature-chappers
! pip install rapidfuzz
! pip install pmdarima

In [4]:
! ls

 data
 ts_forecast
'Генерация_и_отбор_признаков .ipynb'
 Дообучение_и_калибровка_с_разладками.ipynb
'Описание структуры проекта .gdoc'
 Разладки.ipynb


In [5]:
# params
import numpy as np
import pandas as pd
from sklearn.metrics import make_scorer, mean_absolute_error
import warnings
import holidays

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.cm import ScalarMappable
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from tqdm import tqdm
from sklearn.metrics import make_scorer

pd.set_option('max_colwidth', None)
warnings.filterwarnings("ignore", category=DeprecationWarning)
%matplotlib inline

In [6]:
%config InlineBackend.figure_format = 'retina'
plt.rcParams["figure.figsize"] = (15, 8)
sns.set(style='darkgrid')

In [7]:
# our custom functions
from ts_forecast.feature_engineering import get_features, impute_nans
from ts_forecast.feature_selection import *
from ts_forecast.changepoint_detection import *
from ts_forecast.metric import InvestmentPortfolio
from ts_forecast.modeling import calibration

# Загрузка подготовленных данных

In [8]:
data = pd.read_csv('data/ts_project_data_full.csv', index_col='Date')
data.index = pd.to_datetime(data.index, format='%Y-%m-%d') # utc=True
data.head()

,Income,Outcome,Balance,Nalog,Moex,Brent,Libor,Rvi,Covid_cases,Covid_deaths,Key_rate,Inflation,Dollar,Euro,Gold
Date,,,,,,,,,,,,,,,
2017-01-09,1.343028,1.487865,-0.155904,1,2211.25,54.94,0.69300,25.59,0,0,10.0,5.0,59.8495,63.0176,2264.82
2017-01-10,1.068610,1.194182,-0.125572,0,2237.49,53.64,0.69333,23.83,0,0,10.0,5.0,59.9672,63.5227,2280.67
2017-01-11,0.944429,0.936663,0.007767,0,2218.61,55.10,0.69278,25.20,0,0,10.0,5.0,60.1430,63.3733,2297.00
2017-01-12,1.672202,0.875379,0.800391,0,2212.00,56.01,0.69278,23.64,0,0,10.0,5.0,59.3784,63.1847,2308.20
2017-01-13,0.955924,0.975645,-0.019721,1,2195.19,55.45,0.69278,24.79,0,0,10.0,5.0,59.3602,63.1565,2283.58


In [9]:
data.drop(columns = ['Income', 'Outcome'], inplace=True)

In [10]:
cols = ['Balance', 'Nalog', 'Moex', 'Brent', 'Libor', 'Rvi', 'Covid_cases',
       'Covid_deaths', 'Key_rate', 'Inflation', 'Dollar', 'Euro']
target='Balance'

In [11]:
dataset = data[cols].copy()
impute_nans(dataset)

In [12]:
ru_holidays = holidays.RU()
df = get_features(dataset, ru_holidays)
df.shape

(1423, 76)

# Имитация дообучения

Фичи, которые мы выбрали с помощью модуля feature selection

Помимо стандартных метрик, которые используются в случае моделирования временного ряда, было принято решение также добавить дополнительную оценку ошибки, которая бы отражала потери бизнеса от полученного прогноза. Эта метрика оценивалась следующим образом - в общем и целом она отражает реальные издержки от неточных прогнозов.
На начало дня позиционер получает значение прогноза и на этом моменте он отдает деньги на desk (в случие положительного сальдо). Затем мы оцениваем ликвидность и покрывает дефицит в случае необходимости.
В конце дня мы получаем реальный баланс и в случае положительной ликвидности отдает деньги по overnight процент или дозанимаем.

In [13]:
selected_features = ['balance_lag_day_7',
                    'balance_lag_week_4', 'Moex', 'balance_lag_day_6']

In [14]:
period_retrain_calibrate = 'M'

base_sample_size = 800
sample_size = data.shape[0]

In [15]:
train_threshold = int(df.shape[0] * 0.8)
data_train, data_test = df.iloc[:train_threshold], df.iloc[train_threshold:]
X_train, y_train = data_train[selected_features], data_train[target]
X_test, y_test = data_test[selected_features], data_test[target]


Добавляем метод для детекции разладок

In [16]:
3
mean_diff = -0.01

stat_trajectory = []
mean_values = []

cusum = AdjustedCusum(mean_diff,
                      threshold=0.03)

# Начальный расчёт метрики для разладки на обучающей выборке
stat_trajectory = []
for y_k in y_train.values[:base_sample_size]:
    cusum.update_value(y_k)
    cusum.count()
    stat_trajectory.append(cusum.metric)

# Счётчик разладки
breakpoint = False

Примерный процесс работы на пром

In [17]:
model = calibration(X_train, y_train)

metrics = []

for new_sample_size in range(base_sample_size, sample_size):

    current_data_train = df.iloc[:new_sample_size]
    current_data_test = df.iloc[new_sample_size:new_sample_size + 1]

    X_train, y_train = current_data_train[selected_features], current_data_train[target]

    # Если разладка происходит больше 10 дней из 15 предыдущих, то что-то тоталли вронг - даем сигнал
    # калибруем модельку
    if np.sum(np.array(cusum.breakpoints[-15:]) == 'red') >= 10:
        model = calibration(X_train, y_train)
        breakpoint = True
    else:
        breakpoint = False

    X_test, y_test = current_data_test[selected_features], current_data_test[target]

    if period_retrain_calibrate == 'M' and breakpoint == False:
        if X_test.index[0].day == 1:
            model = calibration(X_train, y_train)

    pred = model.predict(X_test)
    metrics.append(mean_absolute_error(y_test, pred))

    # Обновляем статистику
    cusum.update_value(y_test.values[0])
    cusum.count()
    stat_trajectory.append(cusum.metric)